In [2]:
import pandas as  pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [15]:
df = pd.DataFrame({'a': np.random.randint(10,100, size=10),
                   'b': np.random.randint(10,100, size=10),
                   'c': np.random.randint(10,100, size=10)})
df

,a,b,c
0,11,61,97
1,86,60,27
2,28,84,30
3,46,92,74
4,65,43,37
5,42,90,40
6,73,26,72
7,45,91,47
8,69,46,76
9,20,41,44


In [57]:
class StandardScaler(BaseEstimator,TransformerMixin):
    def __init__(self,variables: list|None = None):
        self.variables = variables
        
    def fit(self,X: pd.DataFrame, y = None):
        if self.variables is not None:
            self.numeric_columns_ = pd.Index(self.variables)
        else:
            numeric_columns = X.select_dtypes(include=[np.number]).columns
            self.numeric_columns_ = numeric_columns 
        self.mean_ = X[self.numeric_columns_].mean()
        self.std_ = X[self.numeric_columns_].std()
        return self
    
    def transform(self,X: pd.DataFrame,y=None):
        data = X.copy()
        try:
            data[self.numeric_columns_] = (data[self.numeric_columns_] - self.mean_) / self.std_
        except Exception as e:
            self.errors_ = e
        return data

In [68]:
scaler = StandardScaler(['a','b'])
scaler.fit(df)

StandardScaler(variables=['a', 'b'])

In [69]:
scaler.transform(df)

,a,b,c
0,-1.531144,-0.098451,97
1,1.531144,-0.139472,27
2,-0.837025,0.845039,30
3,-0.102076,1.173209,74
4,0.673703,-0.836834,37
5,-0.265398,1.091166,40
6,1.000347,-1.534196,72
7,-0.142907,1.132188,47
8,0.837025,-0.713770,76
9,-1.163669,-0.918877,44


In [65]:
scaler.errors_

TypeError("unsupported operand type(s) for -: 'str' and 'float'")

In [70]:
scaler.mean_

a    48.5
b    63.4
dtype: float64

In [71]:
pd.concat([scaler.mean_,scaler.std_],axis=1,keys=['mean','std'])

,mean,std
a,48.5,24.491495
b,63.4,24.377585


In [22]:
df

,a,b,c
0,11,61,97
1,86,60,27
2,28,84,30
3,46,92,74
4,65,43,37
5,42,90,40
6,73,26,72
7,45,91,47
8,69,46,76
9,20,41,44
